In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import functional as F
from tqdm import tqdm
from transformers import AutoTokenizer
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.autograd import Variable 
from tqdm import tqdm

In [3]:
class NewsDataset(Dataset):
  def __init__(self, data, tokenizer, max_length):
      self.data = data
      self.tokenizer = tokenizer
      self.max_length = max_length

  def __len__(self):
      return len(self.data)

  def __getitem__(self, index):
      title = self.data.iloc[index]['Title']
      text = self.data.iloc[index]['Text']
      label = self.data.iloc[index]['Category']

      encoding = self.tokenizer.encode_plus(
          title,
          text,
          add_special_tokens=True,
          max_length=self.max_length,
          truncation=True,
          padding='max_length',
          return_tensors='pt'
      )

      input_ids = encoding['input_ids'].squeeze()
      attention_mask = encoding['attention_mask'].squeeze()

      return {
          'input_ids': input_ids,
          'attention_mask': attention_mask,
          'label': label
      }


In [4]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_units, num_classes):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_units, batch_first=True)
        self.fc = nn.Linear(hidden_units, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.lstm(embedded)
        output = self.fc(output[:, -1, :])  # Use the last hidden state
        return output


def train(model, device, train_loader, optimizer, criterion):
    model.train()
    train_loss = 0
    train_acc = 0

    for batch in tqdm(train_loader, desc='Training'):
        input_ids = batch['input_ids'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, dim=1)
        train_acc += (predicted == labels).sum().item()

    train_loss /= len(train_loader.dataset)
    train_acc /= len(train_loader.dataset)

    return train_loss, train_acc


def evaluate(model, device, data_loader, criterion):
    model.eval()
    eval_loss = 0
    eval_f1 = 0

    with torch.no_grad():
        all_labels = []
        all_predictions = []

        for batch in tqdm(data_loader, desc='Evaluation'):
            input_ids = batch['input_ids'].to(device)
            labels = batch['label'].to(device)  # Modify this line

            outputs = model(input_ids)
            loss = criterion(outputs, labels)

            eval_loss += loss.item()
            _, predicted = torch.max(outputs, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

        eval_loss /= len(data_loader.dataset)
        eval_f1 = f1_score(all_labels, all_predictions, average='weighted')

    return eval_loss, eval_f1
     

df = pd.read_csv('/content/data_cleaned.csv')
     

df =df.drop(['Unnamed: 0', 'Title','Text', 'Text_first200',
       'Text_first200_translated', 'Title_Translated'], axis=1)
df = df.rename(columns={"Text_first200_translated_cleaned": "Text", "Title_Translated_cleaned": "Title"})

In [5]:
data = df.copy()

# Encode the labels
label_encoder = LabelEncoder()
data['Category'] = label_encoder.fit_transform(data['Category'])

train_data, val_data = train_test_split(data, test_size=0.15, random_state=42)

# Create the tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define hyperparameters
max_length = 100  # Maximum sequence length for input
vocab_size = tokenizer.vocab_size  # Vocabulary size
embedding_dim = 100  # Embedding dimension
hidden_units = 64
num_classes = len(label_encoder.classes_)  # Number of classes

# Create instances of NewsDataset for training and validation sets
train_dataset = NewsDataset(train_data, tokenizer, max_length)
val_dataset = NewsDataset(val_data, tokenizer, max_length)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


# Instantiate the LSTM-based model
model = LSTMModel(vocab_size, embedding_dim, hidden_units, num_classes)
model.to(device)

# Define the loss criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 40
best_val_loss = float('inf')
best_model_weights = None

for epoch in range(num_epochs):
    # Training
    train_loss, train_f1 = train(model, device, train_loader, optimizer, criterion)
    val_loss, val_f1 = evaluate(model, device, val_loader, criterion)

    # Print epoch results
    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Training Loss: {train_loss:.4f} | Training F1 Score: {train_f1:.4f}')
    print(f'Validation Loss: {val_loss:.4f} | Validation F1 Score: {val_f1:.4f}')
    print('-' * 50)

    # Check if validation loss improved
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_weights = model.state_dict()

# Save the best model
torch.save(best_model_weights, 'lstm_model.pt')


Evaluation: 100%|██████████| 16/16 [00:00<00:00, 16.53it/s]


Epoch 1/40
Training Loss: 0.0518 | Training F1 Score: 0.2934
Validation Loss: 0.0487 | Validation F1 Score: 0.3348
--------------------------------------------------


Evaluation: 100%|██████████| 16/16 [00:00<00:00, 17.54it/s]


Epoch 2/40
Training Loss: 0.0413 | Training F1 Score: 0.4939
Validation Loss: 0.0305 | Validation F1 Score: 0.5873
--------------------------------------------------


Evaluation: 100%|██████████| 16/16 [00:00<00:00, 17.38it/s]


...

Epoch 38/40
Training Loss: 0.0006 | Training F1 Score: 0.9923
Validation Loss: 0.0306 | Validation F1 Score: 0.8140
--------------------------------------------------


Evaluation: 100%|██████████| 16/16 [00:00<00:00, 17.40it/s]


Epoch 39/40
Training Loss: 0.0005 | Training F1 Score: 0.9923
Validation Loss: 0.0308 | Validation F1 Score: 0.8113
--------------------------------------------------


Evaluation: 100%|██████████| 16/16 [00:01<00:00, 11.78it/s]

Epoch 40/40
Training Loss: 0.0005 | Training F1 Score: 0.9934
Validation Loss: 0.0303 | Validation F1 Score: 0.8218
--------------------------------------------------
